### This should lead to the script of the fva result computation
- missing iLC915 citrate cycle reactions

In [6]:
# imports + variables

import cobra
from cobra.flux_analysis import flux_variability_analysis #fva

import yaml
import pandas as pd # load and modify csv data

from importlib import reload


# config preperation
import sys # append path

sys.path.append('../scripts/')
import helperFunction as hf
# reload(hf)

config_name = 'model_config'
config_path = f'../config/{config_name}.yaml'

# load config
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# global variables
_yli_maintenance = (7.8625, 7.8625)
_ppa_maintenance = (2.81, 2.81) # (according to the paper:https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6838487/)
_verbose = False
_loopless = True
_condition = 'loopless_all_glycolysis_TCA_testing_script'



In [2]:
# helper functions
def set_growth_condition(model, biomass_rxn, growth_condition, maintenance_rxn, maintenance):
    """Sets the growth condition of a given model
    @params: model (cobra model), biomass_rxn (str), growth_condition (dict), maintenance_rxn (str), maintenance (tuple of floats)"""
    # set objective function
    model.objective = biomass_rxn
    # set growth condition
    for rxn_id, flux in growth_condition.items():
        model.reactions.get_by_id(rxn_id).bounds = (-flux, flux)
    # set maintenance
    model.reactions.get_by_id(maintenance_rxn).bounds = maintenance

def get_reaction_fluxes(model, reactions, biomass_rxn, growth_condition, maintenance_rxn, maintenance, verbose=False):
    """Computes FBA solution of a given growth condition and returns the fluxes of the reactions of interest"""
    current_fluxes = {}

    # set growth condition
    set_growth_condition(model, biomass_rxn, growth_condition, maintenance_rxn, maintenance)
    # compute FBA solution
    # pfba solution
    pfba_solution = cobra.flux_analysis.pfba(model)
    if verbose:
        print(f'The objective value of the {model} is: {pfba_solution.fluxes[biomass_rxn]}')
    # collect pathway fluxes
    for ec_number, rxn_id in reactions.items():
        if rxn_id == "-":
            continue
        current_fluxes[ec_number] = (pfba_solution.fluxes[rxn_id],rxn_id)
    return current_fluxes

def get_fva_intervals(model, reactions, biomass_rxn, growth_condition, maintenance_rxn, maintenance, loopless=False, fva_solution="",verbose=False):
    """Computes FBA solution of a given growth condition and returns the fluxes of the reactions of interest"""
    current_fva_interval = {}
    
    # set growth condition
    set_growth_condition(model, biomass_rxn, growth_condition, maintenance_rxn, maintenance)
    
    # compute FVA solution if fva_solution == {}
    fva_solution = fva_solution if isinstance(fva_solution, pd.DataFrame) else cobra.flux_analysis.flux_variability_analysis(model, loopless=loopless)
    if verbose:
        print(f'The objective value of the {model} is: {fva_solution.loc[biomass_rxn]}')
    
    # collect pathway intervals
    for ec_number, rxn_id in reactions.items():
        if rxn_id == "-":
            continue
        current_fva_interval[ec_number] = (fva_solution.loc[rxn_id].minimum, fva_solution.loc[rxn_id].maximum)
    return fva_solution, current_fva_interval

def get_fva_flux_table(model_name, fluxes, fva_intervals, reaction_table):
    """Returns a pandas dataframe with fluxes and their corresponding fva intervals."""
    # fluxes to dataframe and key as column
    flux_df = pd.DataFrame.from_dict(fluxes, orient='index', columns=['flux', 'reaction_id'])
    # index to column with name EC
    flux_df.reset_index(inplace=True)
    flux_df.columns = ['EC', f'{model_name}_flux', f'{model_name}_reaction_id']

    # fva_intervals to dataframe
    fva_intervals = pd.DataFrame.from_dict(fva_intervals, orient='index', columns=['min', 'max'])
    # index to column with name EC
    fva_intervals.reset_index(inplace=True)
    fva_intervals.columns = ['EC', f'{model_name}_min', f'{model_name}_max']

    # left join flux_df to glycolysis table on EC
    glycolysis_result = pd.merge(pd.merge(reaction_table, flux_df, on='EC', how='left'), fva_intervals, on='EC', how='left')
    return glycolysis_result

def ppa_prepare_iLC915_model(iLC915_model, verbose=False):
    # prepare iLC915_model
    # prepares iLC915_model if verbose = True: it presents the coefficients of the most important reactions
    print('iLC915 model: ')

    # prepare iLC915_model: needed because of internal cycles and active reactions
    # lower and upper bounds of several reactions should be set to zero according to supplementary material of 
    constraints_2_zero = ['r66','r910','r1104','r239','r111','r106','r490','r791','r243','r252','r253','r307','r308','r404','r405','r1320','r639','r640','r641','r642','r649','r650','r651','r652','r645','r646','r643','r644','r653','r654','r655','r656','r534'] 
    for rxn_id in constraints_2_zero:
        rxn = iLC915_model.reactions.get_by_id(rxn_id)
        rxn.bounds = (0,0)
    # constraint formulate uptake because of not meaningful results
    formulate_uptake = iLC915_model.reactions.get_by_id('r1143')
    formulate_uptake.bounds = (0,0)
    # important carbon sources (biotin, CO2, urea)
    important_carbon = ['r1132','r1137','r1177']

    # all carbon containing exchange reactions
    carbon_exchange_rxns = ['r1122', 'r1123', 'r1124', 'r1126', 'r1127', 'r1129', 'r1130', 'r1131', 'r1132', 'r1134', 'r1135', 'r1137', 'r1138', 'r1139', 'r1140', 'r1141', 'r1144', 'r1145', 'r1146', 'r1147', 'r1148', 'r1149', 'r1151', 'r1152', 'r1153', 'r1154', 'r1155', 'r1156', 'r1157', 'r1158', 'r1161', 'r1162', 'r1163', 'r1165', 'r1167', 'r1168', 'r1170', 'r1172', 'r1173', 'r1174', 'r1175', 'r1176', 'r1177', 'r1178']

    # set all carbon exchange reactions to 0 except important ones
    for rxn_id in carbon_exchange_rxns:
        if rxn_id not in important_carbon:
            rxn = iLC915_model.reactions.get_by_id(rxn_id)
            rxn.bounds = (0,0)

    # check definition of important rxns 
    important_rxns = ['r1133', 'r1141', 'r1145', 'r1144', 'r1148', 'r1160', 'r1137', 'r1188']

    if verbose: 
        for rxn_id in important_rxns:
            rxn = iLC915_model.reactions.get_by_id(rxn_id)
            print(rxn.reactants, rxn.products, rxn.bounds)
            print(rxn_id, rxn.name, hf.formulaWithNames(rxn), 'rxn coefficient: ', rxn.get_coefficient(list(rxn.metabolites.keys())[0]))

    # set objective and maintenance
    biomass_rxn = 'r1339' # biomass
    # set objective: 
    iLC915_model.objective = 'r1339'

    # set maintenance reaction
    maintenance = 'r1188'
    maintenance_rxn = iLC915_model.reactions.get_by_id(maintenance)
    # maintenance_rxn.bounds = (_ppa_maintenance,_ppa_maintenance)
    # maintenance_rxn.bounds = (1,1) # according to iLC915_model
    maintenance_rxn.bounds = (1.2434,1.2434) # according to iLC915_model (1.2434 * 2.26 = 2.81)

    try: 
        iLC915_model.summary()
    except:
        print('iLC915 model: works as expected')

    gluc = iLC915_model.reactions.get_by_id('r1145')
    gluc.bounds = (-10,10)
    try: 
        iLC915_model.summary()
        print('iLC915 model: works as expected')
    except:
        print('iLC915 model: does not work as expected')

In [3]:
# models

# load yli21 model
iYli21_model = cobra.io.read_sbml_model(config['models']['yli21'])

# # load yli20 model
iYli20_model = cobra.io.read_sbml_model(config['models']['yli2.0_corr'])

# # load iNL895 model
iNL895_model = cobra.io.read_sbml_model(config['models']['yliNL895_corr'])

# # load yali4 model
iYali4_model = cobra.io.read_sbml_model(config['models']['yli4_corr'])

# # load iMT1026v3 model
iMT1026v3_model = cobra.io.read_sbml_model(config['models']['ppa1026v3'])

# # load iLC915 model
iLC915_model = cobra.io.read_sbml_model(config['models']['ppaiLC915'])


In [4]:
# reaction connecting dicts + pathways (glycolysis, TCA)
# yli

# Set of EC Numbers:
glycolysis_without_branches = ["2.7.1.1", "5.3.1.9", "2.7.1.11", "4.1.2.13", "5.3.1.1", "1.2.1.12", "2.7.2.3", "5.4.2.11", "4.2.1.11", "2.7.1.40"]


iYli21_glycolysis_without_branches = {
    "2.7.1.1": "R387", # id="R387" name="hexokinase (D-glucose:ATP)"
    "5.3.1.9": "R326", # id="R326" name="glucose-6-phosphate isomerase"
    "2.7.1.11": "R636", # id="R636" name="phosphofructokinase"
    "4.1.2.13": "R313", # id="R313" name="fructose-bisphosphate aldolase"
    "5.3.1.1": "R768", # id="R768" name="triose-phosphate isomerase"
    "1.2.1.12": "R344", # id="R344" name="glyceraldehyde-3-phosphate dehydrogenase"
    "2.7.2.3": "R642", # id="R642" name="phosphoglycerate kinase"
    "5.4.2.11": "R643", # id="R643" name="phosphoglycerate mutase"
    "4.2.1.11": "R292", # id="R292" name="enolase", 
    "2.7.1.40": "R694",# id="R694" name="pyruvate kinase"
}

iYli21_citrate_cycle = {
  '6.4.1.1': 'R690', # NOT MITOCHONRIAL pyruvate carboxylase (Pyruvate -> Oxaloacetate)
  'pyruvate_mitochodrial_transport': 'R1311', # pyruvate mitochondrial transport via proton symport
  '1.1.1.37': 'R533', # (m) malate dehydrogenase (NAD+) (Malate -> Oxaloacetate)
  '1.2.4.1': 'R693', # (m) pyruvate dehydrogenase (Pyruvate -> Acetyl-CoA)
  '2.3.3.1': 'R236', # (m) citrate synthase (Oxaloacetate + Acetyl-CoA -> Citrate)
  '4.2.1.3': 'R238', # (m) aconitase (citrate to cis-aconitate(3-)) # according to Expasy EC = 4.2.1.3
  '1.1.1.41': 'R487', # (m) isocitrate dehydrogenase (NAD+)
  '1.1.1.42': 'R1385', # (m) isocitrate dehydrogenase (NAD+)
  '1.2.4.2': 'R614', # (m) oxoglutarate dehydrogenase (lipoamide)
  '2.3.1.61': 'R613', # (m) oxoglutarate dehydrogenase (dihydrolipoamide S-succinyltransferase)
  '6.2.1.4': 'R741', # (m) succinate-CoA ligase (ADP-forming)
  '6.2.1.5': 'R496', # (m) itaconate-CoA ligase (ADP-forming) id="R496" (?)
  '1.3.5.1': 'R740', # (m) succinate dehydrogenase (ubiquinone-6)
  '4.2.1.2': 'R314', # (m) fumarase (fumarate to malate)
  '1.1.1.37': 'R533', # (m) malate dehydrogenase (malate to oxaloacetate)
}

iYali4_glycolysis_without_branches = {
    "2.7.1.1": "534", # name="hexokinase (D-glucose:ATP)" id="R_534"
    "5.3.1.9": "467", # name="glucose-6-phosphate isomerase" id="R_467"
    "2.7.1.11": "886", # name="phosphofructokinase" id="R_886"
    "4.1.2.13": "450", # name="fructose-bisphosphate aldolase" id="R_450"
    "5.3.1.1": "1054", # name="triose-phosphate isomerase" id="R_1054"
    "1.2.1.12": "486", # name="glyceraldehyde-3-phosphate dehydrogenase" id="R_486"
    "2.7.2.3": "892", # name="phosphoglycerate kinase" id="R_892"
    "5.4.2.11": "893", # name="phosphoglycerate mutase" id="R_893" 
    "4.2.1.11": "366", # name="enolase" id="R_366"
    "2.7.1.40": "962",# name="pyruvate kinase" id="R_962"
}

iYali4_citrate_cycle = {
  '6.4.1.1': '958', # pyruvate carboxylase (id:R_958) (Pyruvate -> Oxaloacetate)
  '1.1.1.37': '713', # malate dehydrogenase (id:R_713) (Malate -> Oxaloacetate)
  'pyruvate_mitochodrial_transport': '2034', # name="pyruvate transport" id="R_2034"
  '1.2.4.1': '961', # name="pyruvate dehydrogenase" id="R_961"
  '1.2.4.1_2': '-', # pyruvate dehydrogenase (lipoamide)
  '2.3.1.12': '-', # pyruvate dehydrogenase (dihydrolipoamide) reversible (mitochondrial) 
  '2.3.3.1': '300', # citrate synthase (mitochondrial) id="R_300"
  '4.2.1.3': '2305', # aconitase (mitochondrial) name="cis-aconitate(3-) to isocitrate" id="R_2305" (?)
  '1.1.1.41': '658', # isocitrate dehydrogenase (NAD+) id="R_658"
  '1.2.4.2': '832', # name="oxoglutarate dehydrogenase (lipoamide)" id="R_832"
  '1.2.4.2_2': '-', # 
  '2.3.1.61': '831', # oxoglutarate dehydrogenase (dihydrolipoamide S-succinyltransferase) id="R_831"
  '6.2.1.4': '1022', # succinate-CoA ligase (ADP-forming)" id="R_1022" 
  '6.2.1.5': '668', # name="itaconate-CoA ligase (ADP-forming)" id="R_668" (?)
  '1.3.5.1': '1021', # succinate dehydrogenase (ubiquinone-6) (m) id="R_1021"
  '4.2.1.2': '451', # fumarase (mitochondrial) id="R_451"
}

# ppa
iMT1026v3_glycolysis_without_branches = {
    "2.7.1.1": "HEX1", # id="R_HEX1" name="hexokinase (D-glucose:ATP)"
    "5.3.1.9": "PGI", # id="R_PGI" name="glucose-6-phosphate isomerase"
    "2.7.1.11": "PFK", # id="R_PFK" name="phosphofructokinase"
    "4.1.2.13": "FBA", # id="R_FBA" name="fructose-bisphosphate aldolase"
    "5.3.1.1": "TPI", # id="R_TPI" name="triose-phosphate isomerase"
    "1.2.1.12": "GAPD", # id="R_GAPD" name="glyceraldehyde-3-phosphate dehydrogenase"
    "2.7.2.3": "PGK", # id="R_PGK" name="phosphoglycerate kinase"
    "5.4.2.11": "PGM", # id="R_PGM" name="phosphoglycerate mutase" # iMT1026v3: EC: 5.4.2.1
    "4.2.1.11": "ENO", # id="R_ENO" name="enolase"
    "2.7.1.40": "PYK",# id="R_PYK" name="pyruvate kinase" 
}

iMT1026v3_citrate_cycle = {
  '6.4.1.1': 'PC', # NOT MITOCHORIAL pyruvate carboxylase (Pyruvate -> Oxaloacetate)
  'pyruvate_mitochodrial_transport': 'PYRt2m', # pyruvate mitochondrial transport via proton symport
  '1.2.4.1': 'PDHa1', # pyruvate dehydrogenase (lipoamide)
  '1.2.4.1': 'PDHa2', # pyruvate dehydrogenase (lipoamide)
  '2.3.3.1': 'CSm', # citrate synthase (mitochondrial)
  '4.2.1.3': 'ACONTm', # aconitase (mitochondrial)
  '1.1.1.41': 'ICDHxm', # isocitrate dehydrogenase (NAD+)
  '1.2.4.2': 'AKGDH1', # 2-oxoglutarate dehydrogenase
  '1.2.4.2': 'AKGDH2', # 2-oxoglutarate dehydrogenase
  '2.3.1.61': 'AKGDbm', # oxoglutarate dehydrogenase (dihydrolipoamide S-succinyltransferase) (mitochondrial)
  '6.2.1.4': 'SUCOASm', # Succinate--CoA ligase (ADP-forming)
  '1.3.5.1': 'SUCD2_u6m', # succinate dehydrogenase (ubiquinone-6), mitochondrial
  '4.2.1.2': 'FUMm', # fumarase (mitochondrial)
  '1.1.1.37': 'MDHm', # malate dehydrogenase, mitochondrial
}

iLC915_glycolysis_without_branches = {
    "2.7.1.1": "r552", # id="R_r552" name="ATP:alpha-D-glucose 6-phosphotransferase"
    "5.3.1.9": "r984", # id="R_r984" name="beta-D-Glucose 6-phosphate ketol-isomerase"
    "2.7.1.11": "r557", # id="R_r557" name="ATP:D-fructose-6-phosphate 1-phosphotransferase"
    "4.1.2.13": "r872", # id="R_r872" name="beta-D-fructose-1,6-bisphosphate D-glyceraldehyde-3-phosphate-lyase" 
    "5.3.1.1": "r978", # id="R_r978" name="D-glyceraldehyde-3-phosphate aldose-ketose-isomerase"
    "1.2.1.12": "r146", # id="R_r146" name="D-glyceraldehyde-3-phosphate:NAD+ oxidoreductase (phosphorylating)" 
    "2.7.2.3": "r596", # id="R_r596" name="ATP:3-phospho-D-glycerate 1-phosphotransferase"
    "5.4.2.11": "r988", # id="R_r988" name="2-Phospho-D-glycerate 2,3-phosphomutase" 
    "4.2.1.11": "r891", # id="R_r891" name="2-phospho-D-glycerate hydro-lyase (phosphoenolpyruvate-forming)" 
    "2.7.1.40": "r576",# id="R_r576" name="ATP:pyruvate 2-O-phosphotransferase"
}

In [8]:
# create fva table

# get flux and fva table for glycolysis, citrate cycle 
# iMT1026v3_citrate_cycle, iYli21_citrate_cycle
# load EC, reaction table with pandas
reaction_table = pd.read_csv(config['reaction_table'], sep=config['seperator'])

# iYali4 results
model_name = 'iYali4'
print(f"{model_name}: \n")
biomass_rxn = 'biomass_C'
maintenance_rxn = 'xMAINTENANCE'
growth_condition = {
    '1714': 100, # glucose
    '1709': 0, # fructose
    '1808': 0, # glycerol
}

# For glycolysis and citrate cycle
iYali4_glycolysis_and_citrate_cycle = {**iYali4_glycolysis_without_branches, **iYali4_citrate_cycle}

current_fluxes = get_reaction_fluxes(iYali4_model, iYali4_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _yli_maintenance)
iYali4_fva_solution, current_fva_intervals = get_fva_intervals(iYali4_model, iYali4_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _yli_maintenance, _loopless, "", _verbose)
# add both to the csv table
reaction_table = get_fva_flux_table(model_name, current_fluxes, current_fva_intervals, reaction_table)
# output
# reaction_table.to_csv(f"{config['results']['metabolic_fluxes']}/{_condition}_flux_fva.csv", sep=config['seperator'], index=False)

# # iYli647 results
# model_name = "iYLI647"
# print(f"{model_name}: \n")
# biomass_rxn = 'biomass_C'
# maintenance_rxn = 'ATPM'
# growth_condition = {
#     'EX_glc(e)': 100, # glucose
#     'EX_fru(e)': 0, # fructose
#     'EX_glyc(e)': 0, # glycerol
# }

# # For glycolysis and citrate cycle
# iYli647_glycolysis_and_citrate_cycle = {**iYli647_glycolysis_without_branches, **iYli647_citrate_cycle}

# current_fluxes = get_reaction_fluxes(iYli647_model, iYli647_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _yli_maintenance)
# iYli647_fva_solution, current_fva_intervals = get_fva_intervals(iYli647_model, iYli647_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _yli_maintenance, _loopless, "", _verbose)
# # add both to the csv table
# reaction_table = get_fva_flux_table(model_name, current_fluxes, current_fva_intervals, reaction_table)
# reaction_table.to_csv(f"{config['results']['metabolic_fluxes']}/{_condition}_flux_fva.csv", sep=config['seperator'], index=False)

# # iMT1026v3 results
model_name = 'iMT1026v3'
print(f"{model_name}: \n")
biomass_rxn = 'growth' # BIOMASS_glyc # (id:BIOMASS) Biomass composition # growth biomass_c --> biomass_e
maintenance_rxn = 'ATPM' # 0.55 
growth_condition = {
    'Ex_glc_D': 100, # glucose
    'Ex_fru': 0, # fructose
    'Ex_glyc': 0, # glycerol
    'Ex_o2': 1000, # O2
}

# For glycolysis and citrate cycle
iMT1026v3_glycolysis_and_citrate_cycle = {**iMT1026v3_glycolysis_without_branches, **iMT1026v3_citrate_cycle}
# fluxes and fva intervals
current_fluxes = get_reaction_fluxes(iMT1026v3_model, iMT1026v3_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _ppa_maintenance)
iMT1026v3_fva_solution, current_fva_intervals = get_fva_intervals(iMT1026v3_model, iMT1026v3_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _ppa_maintenance, _loopless, "", _verbose)
# get flux table
reaction_table = get_fva_flux_table(model_name, current_fluxes, current_fva_intervals, reaction_table)
# output
# reaction_table.to_csv(f"{config['results']['metabolic_fluxes']}/{_condition}_flux_fva.csv", sep=config['seperator'], index=False)

############ !!! In order to get included the citrate cycle connection is required !!!! #################
# # iLC915 results
# model_name = 'iLC915'
# print(f"{model_name}: \n")
# biomass_rxn = 'r1339' # r1339 Growth # r1187 biomass formation
# maintenance_rxn = 'r1188' # Maintenance
# growth_condition = {
#     'r1145': 100, # glucose
#     'r1144': 0, # fructose
#     'r1148': 0, # glycerol
#     'r1160': 1000, # O2
# }

# # For glycolysis and citrate cycle
# iLC915_glycolysis_and_citrate_cycle = {**iLC915_glycolysis_without_branches, **iLC915_citrate_cycle}
# # fluxes and fva intervals
# current_fluxes = get_reaction_fluxes(iLC915_model, iLC915_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _ppa_maintenance)
# iLC915_fva_solution, current_fva_intervals = get_fva_intervals(iLC915_model, iLC915_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _ppa_maintenance, _loopless, "", _verbose)
# get flux table
reaction_table = get_fva_flux_table(model_name, current_fluxes, current_fva_intervals, reaction_table)
# output
# reaction_table.to_csv(f"{config['results']['metabolic_fluxes']}/{_condition}_flux_fva.csv", sep=config['seperator'], index=False)



# iYli21 results
model_name = 'iYli21'
print(f"{model_name}: \n")
biomass_rxn = 'biomass_C'
maintenance_rxn = 'xMAINTENANCE'
growth_condition = {
    'R1070': 100, # glucose
    'R1065': 0, # fructose
    'R1141': 0, # glycerol
}
# For glycolysis and citrate cycle
iYli21_glycolysis_and_citrate_cycle = {**iYli21_glycolysis_without_branches, **iYli21_citrate_cycle}

current_fluxes = get_reaction_fluxes(iYli21_model, iYli21_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _yli_maintenance)
iYli21_fva_solution, current_fva_intervals = get_fva_intervals(iYli21_model, iYli21_glycolysis_and_citrate_cycle, biomass_rxn, growth_condition, maintenance_rxn, _yli_maintenance, _loopless, "", _verbose)
# add both to the csv table
reaction_table = get_fva_flux_table(model_name, current_fluxes, current_fva_intervals, reaction_table)
# reaction_table.to_csv(f"{config['results']['metabolic_fluxes']}/{_condition}_flux_fva.csv", sep=config['seperator'], index=False)



iYali4: 

iMT1026v3: 



/Users/ampholyt/Coding/BSEP22/Code/With_Bockmayr/EFM_calculation_cobamp/venv/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/Users/ampholyt/Coding/BSEP22/Code/With_Bockmayr/EFM_calculation_cobamp/venv/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


iYli21: 



/Users/ampholyt/Coding/BSEP22/Code/With_Bockmayr/EFM_calculation_cobamp/venv/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/Users/ampholyt/Coding/BSEP22/Code/With_Bockmayr/EFM_calculation_cobamp/venv/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
